# Running This Notebook

To run this notebook, you will need to have [tensorflow](https://www.tensorflow.org/install/docker) installed. When installing tensorflow, it is optional whether or not you want to use GPU to significantly speed up computations. This is not absolutely necessary to simply run this notebook, the code will run fine, but highly recomended if you plan on running the training stage of the model. You may get warning messages from Tensorflow saying that Cuda (GPU) is not enabled, but you can ignore those.

I reccomend running this notebook in [this](https://hub.docker.com/r/tensorflow/tensorflow/) docker environment, with jupyter and GPU enabled.

In [ ]:
import os
import time
import numpy as np
from PIL import Image
import tensorflow as tf
import keras.backend as K
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras.initializers import RandomNormal

# Load Data

When tensorflow reads images, it likes to keep them in tensors that are batches of images. Batches are used to speed up the training of models. The tensors from batches are larger, but fewer tensor operations need to be done.

Also note, that unlike numpy arrays, tensorflow data sets are not directly stored in memory, as they are sometimes too big. Instead they specify a data pipeline.

In [ ]:
#Function for reading data from our image directories
def getBatchDataset(imageDir, imageHeight, imageWidth, batch_size):
    return tf.keras.utils.image_dataset_from_directory(
        imageDir,
        labels=None,
        seed=123,
        image_size=(imageHeight, imageWidth),
        batch_size=batch_size)

#Image parameters
batch_size = 4
imageWidth = 256
imageHeight = 512

#Choose image data directory depending whether we want to use all data, or a small subset for debugging.
useAllData = False
if useAllData:
    featuresDir = './Data/ImageTrainingData/Features'
    targetsDir = './Data/ImageTrainingData/Targets'
else:
    featuresDir = './Data/SmallImageTrainingDataset/Features'
    targetsDir = './Data/SmallImageTrainingDataset/Targets'

#Load data, extract a batch of each dataset, and extract an image from each batch
featureData = getBatchDataset(featuresDir, imageHeight, imageWidth, batch_size)
targetData = getBatchDataset(targetsDir, imageHeight, imageWidth, batch_size)

featureBatch = next(iter(featureData))
targetBatch = next(iter(targetData))

featureImage = featureBatch[2].numpy()
targetImage = targetBatch[2].numpy()

#Plot Training Data
fig, axis = plt.subplots(1, 2)
fig.set_figheight(10)
fig.set_figwidth(10)
axis[0].imshow(featureImage.astype(np.uint8))
axis[1].imshow(targetImage.astype(np.uint8))
axis[0].set_title('Current Iteration')
axis[1].set_title('Next Iteration')
axis[0].set_xticks([])
axis[0].set_yticks([])
axis[1].set_xticks([])
axis[1].set_yticks([])
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

# First Attempt at Making a GAN

Here we walk through our first attempt at making a Generative Adversarial Network to simulate erosion. A good amount of this section is based on [this GAN tutorial for tensorflow](https://www.tensorflow.org/tutorials/generative/dcgan), but is modified to suit our data.

Although the first attempt gave terrible results, it did help me learn the basics of making a GAN and helped me be more familiar with more advanced tensorflow models. I'll keep the code here amyways, to demonstrate the progress of this project.

### Generator Model

The generator model bellow is based the tensorflow tutorial that I was following, with some adjustments made to better suit it our data, and further changes made based on trial and error.

In [ ]:
#Here we define the structure of the generator model
def getGeneratorModel(imageWidth, imageHeight):
    model = tf.keras.Sequential()
    
    #A few convolution layers for interpreting input image
    model.add(layers.Conv2D(64, 4, activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(32, 3, activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())
    
    model.add(layers.Dense(16 * 8 * 3, activation='relu'))
    model.add(layers.Reshape((16, 8, 3)))
    
    #Some layers for generating images
    model.add(layers.Conv2DTranspose(64, (10, 10), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(32, (10, 10), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(16, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(8, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    #Final output layer
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

As of now, our model has not been trained yet, so it's output should be a noisy image.

In [ ]:
#Generate some random results with our untrained model
batch = next(iter(featureData))
generator = getGeneratorModel(imageWidth, imageHeight)
generatedImageBatch = generator(batch, training=False)

#Check that the output images has the desired dimensions
print(batch.shape)
print(generatedImageBatch.shape)

image = generatedImageBatch[2].numpy()
image /= np.max(image)
image *= 255

#Visualize results
plt.figure(figsize=(10, 8))
plt.imshow(image.astype(np.uint8))
plt.show()

### Discriminator Model

The discriminator model will try to distinguish between generated and fake images. It returns a value between $[-1, 1]$ depending on how confident it is that the image is real $1$ or fake $-1$ (created by our generator).

In [ ]:
def getDiscriminatorModel(imageWidth, imageHeight):
    model = tf.keras.Sequential()
    
    #Two convolutional layers
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    #Output either a 1 or 0 for real or fake
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

Using our image generated from our untrained generator network, we test that our untrained discriminator model is working.

In [ ]:
discriminator = getDiscriminatorModel(imageWidth, imageHeight)
decision = discriminator(generatedImageBatch)
print(decision[2])

### Loss Functions

Loss functions in machine learning are used to measure how well a particular model is performing. During the training stage, our model will adjust it's parameters in an attempt to minimize it's loss function. We will need a loss function for both the generator and discriminator models.

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

#Checkpoints incase our model training is interrupted
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

### Defining the Training Loop

Here we specify the training loop

In [ ]:
#Training loop of GAN
@tf.function
def train_step(inputBatch, outputBatch):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        #Generate next simulation iterator using our generator model
        generatedImages = generator(inputBatch, training=True)
        
        #Pass real and fake images to discriminator
        real_output = discriminator(outputBatch, training=True)
        fake_output = discriminator(generatedImages, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(featureData, targetData, epochs, generator):
    testBatch = next(iter(featureData))
    
    for epoch in range(epochs):
        start = time.time()
        for inputBatch, outputBatch in zip(featureData, targetData):
            train_step(inputBatch, outputBatch, generator)

        # Produce images for the GIF as you go
        generate_and_save_images(generator, epoch + 1, testBatch)

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    generate_and_save_images(generator, epochs, testBatch)

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    plt.imshow(predictions[0].numpy())
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))

Change to true to train the model. If you don't feel like waiting for the training to finish however, we provide some results images bellow. As we can see, the inital results after our first attempt are really bad, and does not show erosion at all.

In [ ]:
if False:
    epochs = 5
    train(featureData, targetData, epochs)

Example results image:
<div>
<img src="Data/ResultsImages/Trial1/image_at_epoch_0003.png" width="900">
</div>

As we can see, our first attempt gave us really bad results.

# GAN Model Based on Eric Guerin's Paper

### Generator Model
After testing, the above model did not give desirable results. To fix this, we will see how the following GAN by [Eric Guerin et. al.](https://hal.archives-ouvertes.fr/hal-01583706/file/tog.pdf) performs on our data. We will modify various parameters and layers to make the GAN work with our data.

- Just like our above model, they implement a encoder layer using a sequence of convolutional layers, and an decoder with a sequence of deconvolutional layers.
- In addition, they also use skip layers to connect the convolutional layers with deconvolutional layers of the same resolution.
- Their graphics card has 12 Gb of dedicated graphics RAM, I only have 8 Gb, so I may have to work around this. I should still be able to get reasonable results though.

The code bellow is copied and pasted from the [*nanoxas/sketch-to-terrain* Github](https://github.com/nanoxas/sketch-to-terrain/blob/master/model.py), and adjusted for our purposes. The original code is the same code provided by Eric Guerin et. al's paper.

In [ ]:
#Here we define the structure of the generator model
def getInspiredGeneratorModel(imageWidth, imageHeight):
    
    #Input layer
    inputs = layers.Input((imageHeight, imageWidth, 3))
    
    #A few convolutional layers with maxpooling
    #This is the encoder part of the model that interprets the input image
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    
    #Their last convolution layer in their encoder seems to have skip layer with noise introduced to it
    #Noise is often used to avoid the overfiting of a machine learning model
    noise = layers.Input((K.int_shape(conv5)[1], K.int_shape(conv5)[2], K.int_shape(conv5)[3]))
    conv5 = layers.Concatenate()([conv5, noise])
    
    #From my understanding, upsampling is used to increase the weight of data in the minority class
    #Skip layer (connects conv4 layer directly to up6 layer), and more convolutional layers
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv5))
    merge6 = layers.Concatenate()([conv4, up6])
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.Concatenate()([conv3, up7])
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.Concatenate()([conv2, up8])
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.Concatenate()([conv1, up9])
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv9)
    conv10 = layers.Conv2D(3, 1, activation='tanh')(conv9)
    
    #Create and return the final model
    model = Model(inputs=[inputs, noise], outputs=conv10)
    return model

#Generate some random results with our untrained model
batch = next(iter(featureData))
generator = getInspiredGeneratorModel(imageWidth, imageHeight)
generator.summary()
noise = np.random.normal(0, 1, (batch.shape[0], 32, 16, 1024))
generatedImageBatch = generator([batch, noise], training=False)

#Check that the output images has the desired dimensions
print(batch.shape)
print(generatedImageBatch.shape)

From the output bellow, we can clearly see the output of the untrained Unet model above maintains many of the features of the input data, which is definetely something we will want. Note that considering that the model has not been trained yet, the results already look much more promising compared to our first attempt.

In [ ]:
#Plot both input and output image examples of our model
inputImage = batch[2].numpy()
outImage = generatedImageBatch[2].numpy() * 256

fig, axis = plt.subplots(1, 2)
fig.set_figheight(10)
fig.set_figwidth(10)
axis[0].imshow(inputImage.astype(np.uint8))
axis[1].imshow(outImage.astype(np.uint8))
axis[0].set_title('Input Image')
axis[1].set_title('Output Image')
axis[0].set_xticks([])
axis[0].set_yticks([])
axis[1].set_xticks([])
axis[1].set_yticks([])
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

In order to read the output images as data, we want to avoid any borders associated with the above *plt.imshow()* plot. The code bellow lets us save the data as appropriate images.

In [ ]:
#Code for outputing results as an image (and not an imshow plot)
image = generatedImageBatch[2].numpy() * 256
img = Image.fromarray(image.astype(np.uint8))
img.show()

## Discussion

Note that the above images are the results of passing our input data through the yet un-trained generator model. The model currently has 31,050,243 trainable parameters, which are initiated by random numbers. Notice how the output images do indeed have a resamblance to the input images, which is a result of the skip layers. This resamblance leads me to believe that the model is clearly capable of learning our training data.

From the code bellow, we can see that passing a batch of 32 images through our model takes about 0.015 seconds using my GTX 2080 GPU. This means we can expect our final algorithm (after training) to take less than half a milisecond per iteration to simulate erosion. If this works as intended, then this will be by far the fastest erosion algorithm that I am aware of, by multiple orders of magnitude.

Note that if your installation of tensorflow does not have access to a GPU, then your code might take significantly longer to run this cell with a CPU instead. For me, a batch of 32 images takes about 27 seconds to run on my CPU, less than 1 second per iteration.

In [ ]:
start = time.time()
noise = np.random.normal(0, 1, (batch.shape[0], 32, 16, 1024))
generatedImageBatch = generator([batch, noise], training=False)
print(time.time() - start)

The code bellow is the original code from [Eric Guerin et. al.'s](https://github.com/nanoxas/sketch-to-terrain/blob/master/model.py) github.

<details>
<summary> 
    <p style="font-style: italic; color:blue;">
        Click here to see original code
    </p>
</summary>
<br> <p style="font-style: italic;">

    from keras.layers import *
    from keras.models import Model
    from keras.initializers import RandomNormal
    import keras.backend as K

    def UNet(shape):

        inputs = Input(shape)
        conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
        conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
        conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
        conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
        conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
        conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
        noise = Input((K.int_shape(conv5)[1], K.int_shape(conv5)[2], K.int_shape(conv5)[3]))
        conv5 = Concatenate()([conv5, noise])

        up6 = Conv2D(
            512,
            2,
            activation='relu',
            padding='same')(
            UpSampling2D(
                size=(
                    2,
                    2))(conv5))
        merge6 = Concatenate()([conv4, up6])
        conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
        conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

        up7 = Conv2D(
            256,
            2,
            activation='relu',
            padding='same')(
            UpSampling2D(
                size=(
                    2,
                    2))(conv6))
        merge7 = Concatenate()([conv3, up7])
        conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
        conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

        up8 = Conv2D(
            128,
            2,
            activation='relu',
            padding='same')(
            UpSampling2D(
                size=(
                    2,
                    2))(conv7))
        merge8 = Concatenate()([conv2, up8])
        conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
        conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

        up9 = Conv2D(
            64,
            2,
            activation='relu',
            padding='same')(
            UpSampling2D(
                size=(
                    2,
                    2))(conv8))
        up9 = ZeroPadding2D(((0, 1), (0, 1)))(up9)
        merge9 = Concatenate()([conv1, up9])
        conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
        conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
        conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)
        conv10 = Conv2D(1, 1, activation='tanh')(conv9)

        model = Model(input=[inputs, noise], output=conv10)
        model.summary()
        return model

</p></details>

### Discriminator Model

**The following section is not finished**

Similarly, we will use a discriminator model taken from Eric Guerin's paper, and see how well it performs. According to their paper, their discriminator model returns decisions for multiple patches of a single input image. Which is a bit different to what we had before.

In [ ]:
def getInspiredDiscriminatorModel(imageHeight, imageWidth):
    
    #Create inputs of initial and generated image to discrimate, and combine them
    init = RandomNormal(stddev=0.02)
    initialImage = layers.Input(shape=(imageHeight, imageWidth, 3))
    generatedImage = layers.Input((imageHeight, imageWidth, 3))
    combinedImages = layers.Concatenate()([initialImage, generatedImage])
    
    #Main structure of the discriminator neural network model
    d = layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(combinedImages)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(512, (4, 4), padding='same', kernel_initializer=init)(d)
    x = layers.LeakyReLU(alpha=0.2)(d)
    output = layers.Conv2D(1, (4, 4), padding='same', activation='sigmoid', kernel_initializer=init)(d)
    
    #Create the model and return it
    model = Model([initialImage, generatedImage], output)
    return model

We test that the discriminator model gives a desirable output.

In [ ]:
generator = getInspiredGeneratorModel(imageWidth, imageHeight)
discriminator = getInspiredDiscriminatorModel(imageHeight, imageWidth)
#discriminator.summary()

batch = next(iter(featureData))
noise = np.random.normal(0, 1, (batch.shape[0], 32, 16, 1024))
generatedImageBatch = generator([batch, noise], training=False)
decision = discriminator((batch, generatedImageBatch))

print(decision.shape)
print(batch.shape)

Their code also contains the following third model, which was not mentioned in their paper. My best guess is that it combines both generator and discriminator models into one combined tensorflow model.

In [ ]:
def mount_discriminator_generator(generator, discriminator, image_shape):
    discriminator.trainable = False
    input_gen = layers.Input(shape=image_shape)
    input_noise = layers.Input(shape=(32, 16, 1024))
    gen_out = generator([input_gen, input_noise])
    output_d = discriminator([gen_out, input_gen])
    model = Model(inputs=[input_gen, input_noise], outputs=[output_d, gen_out])
    return model

#Create a mountDiscriminator and print summary of model
mountDiscriminator = mount_discriminator_generator(generator, discriminator, (imageHeight, imageWidth, 3))
mountDiscriminator.summary()

# Training Loop According to Paper

The code bellow defines the GAN training loop based on the paper that I'm following. Note that we will output images generated by the GAN during the training stage to help visualize the learning process.

In [ ]:
#Convert TF batchdataset into a numpy array
def batchDatasetToNumpy(dataset):
    dataNP = []
    for batch in dataset:
        for dat in batch:
            dataNP.append(dat.numpy())
    dataNP = np.array(dataNP)
    dataNP /= np.max(dataNP)
    return dataNP

#Randomly select a portion of feature data along with their corresponding target data
def generateRealSamples(features, targets, n_samples, nXPatches, nYPatches):
    idx = np.random.randint(0, features.shape[0], n_samples)
    feats = features[idx]
    targs = targets[idx]
    desiredDiscriminatorProbs = np.ones((n_samples, nXPatches, nYPatches, 1))
    return feats, desiredDiscriminatorProbs, targs

#Generate fake images corresponding to the real targets
def generateFakeSamples(generator, dataset, nXPatches, nYPatches):
    w_noise = np.random.normal(0, 1, (dataset.shape[0], 32, 16, 1024))
    fakeOutput = generator.predict([dataset, w_noise])
    desiredDescriminatorProbs = np.zeros((len(fakeOutput), nXPatches, nYPatches, 1))
    return fakeOutput, desiredDescriminatorProbs

#Using the test feature image, we save generator output images to view the progress of it's training
def saveProgressImage(generator, testImage, iteration, dirName='./ProgressImages/Progress{}.png'):
    w_noise = np.random.normal(0, 1, (1, 32, 16, 1024))
    generatedImage = generator([firstImage[np.newaxis], w_noise], training=False)[0]
    generatedImage = generatedImage.numpy()
    generatedImage -= np.min(generatedImage)
    generatedImage /= np.max(generatedImage)
    generatedImage *= 256

    img = Image.fromarray(generatedImage.astype(np.uint8))
    img.save(dirName.format(iteration))

#Convert data sets to numpy arrays
featureNP = batchDatasetToNumpy(featureData)
targetNP = batchDatasetToNumpy(targetData)

#Chose a random selection of images for this batch, and generate required data for training
feats, desiredDiscriminatorProbs, targs = generateRealSamples(featureNP, targetNP, batch_size, 32, 16)
fakeOutput, desiredDescriminatorProbs = generateFakeSamples(generator, feats, 32, 16)

print(feats.shape)
print(targs.shape)
print(fakeOutput.shape)
print(desiredDiscriminatorProbs.shape)

In [ ]:
#Image parameters
batch_size = 4
imageWidth = 256
imageHeight = 512

#Choose image data directory depending whether we want to use all data, or a small subset for debugging.
useAllData = True
if useAllData:
    featuresDir = './Data/ImageTrainingData/Features'
    targetsDir = './Data/ImageTrainingData/Targets'
else:
    featuresDir = './Data/SmallImageTrainingDataset/Features'
    targetsDir = './Data/SmallImageTrainingDataset/Targets'

#Load data and extract a batch of each dataset
featureData = getBatchDataset(featuresDir, imageHeight, imageWidth, batch_size)
targetData = getBatchDataset(targetsDir, imageHeight, imageWidth, batch_size)
featureNP = batchDatasetToNumpy(featureData)
targetNP = batchDatasetToNumpy(targetData)
featureBatch = next(iter(featureData))
targetBatch = next(iter(targetData))

#Adam optimizer is a popular algorithm for training neural network models with
adamOptimizer = tf.keras.optimizers.Adam(1e-4)

#Get and setup generator, discriminator and composite models
generator = getInspiredGeneratorModel(imageWidth, imageHeight)
discriminator = getInspiredDiscriminatorModel(imageHeight, imageWidth)
discriminator.compile(loss='binary_crossentropy', optimizer=adamOptimizer)
compositeModel = mount_discriminator_generator(generator, discriminator, (imageHeight, imageWidth, 3))
compositeModel.compile(loss=['binary_crossentropy', 'mae'], loss_weights=[1, 3], optimizer=adamOptimizer)


nEpochs = 5 #Number of times to repeat overall training loop
batchSize = 4 #n_batch in original code
batchesPerEpochs = int(len(featureData) / batchSize)
nSteps = nEpochs * batchesPerEpochs

avg_loss = 0
avg_dloss = 0

#Get number of patches in X and Y directions generated by our discriminator
batch = next(iter(featureData))
w_noise = np.random.normal(0, 1, (batch_size, 32, 16, 1024))
generatedImageBatch = generator([batch, w_noise], training=False)
decision = discriminator((batch, generatedImageBatch))
nXPatches, nYPatches = decision.shape[1], decision.shape[2]

#Save a generated image to track progress before training
firstImage = featureNP[0]
saveProgressImage(generator, firstImage, 0)

#Main training loop
if False:
    for i in range(nSteps):

        #Chose a random selection of images for this batch, and generate required data for training
        feats, realLabels, targs = generateRealSamples(featureNP, targetNP, batchSize, nXPatches, nYPatches)
        fakeOutput, fakeLabels = generateFakeSamples(generator, feats, nXPatches, nYPatches)

        w_noise = np.random.normal(0, 1, (batch_size, 32, 16, 1024))
        losses_composite = compositeModel.train_on_batch([feats, w_noise], [realLabels, targs])

        loss_discriminator_fake = discriminator.train_on_batch([fakeOutput, feats], fakeLabels)
        loss_discriminator_real = discriminator.train_on_batch([targs, feats], realLabels)

        d_loss = (loss_discriminator_fake + loss_discriminator_real) / 2
        avg_dloss = avg_dloss + (d_loss - avg_dloss) / (i + 1)
        avg_loss = avg_loss + (losses_composite[0] - avg_loss) / (i + 1)
        print('total loss:' + str(avg_loss) + ' d_loss:' + str(avg_dloss))

        #Save progress images of the generator output
        if i % 1 == 0:
            firstImage = featureNP[0]
            saveProgressImage(generator, firstImage, i)

Results of the above GAN seem promising, I will probably test using it with an improved data set generated in the next notebook.

# Using Simpler Loss Functions

[In this paper](https://arxiv.org/pdf/1810.08217.pdf), the authors used the same U-Net generator model as Eric does in his GAN. They used the U-Net model to simulate fluid/air simulations, but trained it with a simple $L_1$-Norm loss function rather than using a discriminator. To replicate this, we can re-use eric's Unet generator model, but use a simple loss function rather than training a dscrimator model along with it.

In [ ]:
#Here we define the structure of the generator model
def getInspiredGeneratorModel(imageWidth, imageHeight):
    
    #Input layer
    inputs = layers.Input((imageHeight, imageWidth, 3))
    
    #A few convolutional layers with maxpooling
    #This is the encoder part of the model that interprets the input image
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    
    #Their last convolution layer in their encoder seems to have skip layer with noise introduced to it
    #Noise is often used to avoid the overfiting of a machine learning model
    noise = layers.Input((K.int_shape(conv5)[1], K.int_shape(conv5)[2], K.int_shape(conv5)[3]))
    conv5 = layers.Concatenate()([conv5, noise])
    
    #From my understanding, upsampling is used to increase the weight of data in the minority class
    #Skip layer (connects conv4 layer directly to up6 layer), and more convolutional layers
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv5))
    merge6 = layers.Concatenate()([conv4, up6])
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.Concatenate()([conv3, up7])
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.Concatenate()([conv2, up8])
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.Concatenate()([conv1, up9])
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv9)
    conv10 = layers.Conv2D(3, 1, activation='tanh')(conv9)
    
    #Create and return the final model
    model = Model(inputs=[inputs, noise], outputs=conv10)
    #model = Model(inputs=[inputs], outputs=conv10)
    return model

In [ ]:
#Image parameters
batch_size = 4
imageWidth = 256
imageHeight = 512

#Choose image data directory depending whether we want to use all data, or a small subset for debugging.
useAllData = False
if useAllData:
    featuresDir = './ImageTrainingData/Features'
    targetsDir = './ImageTrainingData/Targets'
else:
    featuresDir = './SmallImageTrainingDataset/Features'
    targetsDir = './SmallImageTrainingDataset/Targets'


#Load data and extract a batch of each dataset
featureData = getBatchDataset(featuresDir, imageHeight, imageWidth, batch_size)
targetData = getBatchDataset(targetsDir, imageHeight, imageWidth, batch_size)
featureNP = batchDatasetToNumpy(featureData)
targetNP = batchDatasetToNumpy(targetData)


#U-Net generator model
generator = getInspiredGeneratorModel(imageWidth, imageHeight)
mseLoss = tf.keras.losses.MeanSquaredError()
generator.compile(optimizer='adam', loss=mseLoss, metrics=['mean_absolute_error'])

#Save a generated image to track progress before training
firstImage = featureNP[0]
saveProgressImage(generator, firstImage, 0)

#Main training loop
for i in range(nSteps):
    
    #Chose a random selection of images for this batch, and generate required data for training
    feats, realLabels, targs = generateRealSamples(featureNP, targetNP, batchSize, nXPatches, nYPatches)
    #fakeOutput, fakeLabels = generateFakeSamples(generator, feats, nXPatches, nYPatches)
    
    w_noise = np.random.normal(0, 1, (batch_size, 32, 16, 1024))
    losses = generator.train_on_batch([feats, w_noise], y=targs)
    
    print('Losses: {}'.format(losses))
    
    #Save progress images of the generator output
    if i % 1 == 0:
        firstImage = featureNP[0]
        saveProgressImage(generator, firstImage, i)

In [ ]:
#Using the test feature image, we save generator output images to view the progress of it's training
def saveProgressImage(generator, testImage, iteration, dirName='./ProgressImages/Progress{}.png'):
    w_noise = np.random.normal(0, 1, (1, 32, 16, 1024))
    generatedImage = generator([firstImage[np.newaxis], w_noise], training=False)[0]
    generatedImage = generatedImage.numpy()
    generatedImage -= np.min(generatedImage)
    generatedImage /= np.max(generatedImage)
    generatedImage *= 256

    img = Image.fromarray(generatedImage.astype(np.uint8))
    img.save(dirName.format(iteration))

firstImage = featureNP[0]
saveProgressImage(generator, firstImage, 0)

In [ ]:
firstImage = featureNP[0]

w_noise = np.random.normal(0, 1, (1, 32, 16, 1024))
generatedImage = generator([firstImage[np.newaxis], w_noise], training=False)[0]

print(generatedImage.shape)
print(np.max(generatedImage))
print(np.min(generatedImage))
print(np.max(firstImage))

plt.imshow(generatedImage)

In [ ]:
#Using the test feature image, we save generator output images to view the progress of it's training
def saveProgressImage(generator, testImage, iteration, dirName='./ProgressImages/Progress{}.png'):
    generatedImage = generator(firstImage[np.newaxis], training=False)[0]
    generatedImage = generatedImage.numpy()
    generatedImage -= np.min(generatedImage)
    generatedImage /= np.max(generatedImage)
    generatedImage *= 256

    img = Image.fromarray(generatedImage.astype(np.uint8))
    img.save(dirName.format(iteration))

firstImage = featureNP[0]
saveProgressImage(generator, firstImage, 1)

In [ ]:
firstImage = featureNP[0]
generatedImage = generator(firstImage[np.newaxis], training=False)[0]
generatedImage = generatedImage.numpy()
print(np.min(generatedImage, axis=2).shape)

generatedImage -= np.min(generatedImage)
generatedImage /= np.max(generatedImage)
generatedImage *= 256

img = Image.fromarray(generatedImage.astype(np.uint8))
img.show()

Loss functions for measuring how well our models are performing.

In [ ]:
#Loss function for the discriminator model
def inspiredDiscriminatorLoss(realOutput, fakeOutput):
    crossEntropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    realLoss = crossEntropy(tf.ones_like(realOutput), realOutput)
    fakeLoss = crossEntropy(tf.zeros_like(fakeOutput), fakeOutput)
    totalLoss = realLoss + fakeLoss
    return totalLoss

#Loss function for the generator model
def inspiredGeneratorLoss(fakeOutput):
    crossEntropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    return crossEntropy(tf.ones_like(fakeOutput), fakeOutput)

Make sure that our loss functions are return results of a reasonable type.

In [ ]:
#Load data and extract a batch
featureData = getBatchDataset(featuresDir, imageHeight, imageWidth, batch_size)
targetData = getBatchDataset(targetsDir, imageHeight, imageWidth, batch_size)
featureBatch = next(iter(featureData))
targetBatch = next(iter(targetData))

#Generate next simulation iterator using our currently untrained generator model
generator = getInspiredGeneratorModel(imageWidth, imageHeight)
generatedImageBatch = generator(featureBatch, training=False)
ones = tf.ones_like(generatedImageBatch)

#Pass real and fake images to discriminator
real_output = discriminator((featureBatch, targetBatch), training=False)
fake_output = discriminator((featureBatch, generatedImageBatch), training=False)

#Get losses
discriminatorLoss = inspiredDiscriminatorLoss(real_output, fake_output)
generatorLoss = inspiredGeneratorLoss(fake_output)



print(generatorLoss.numpy())
print(discriminatorLoss.numpy())

image = generatedImageBatch[0].numpy()
oneImage = ones[0].numpy()

plt.imshow(oneImage)

In [ ]:
#Define the training loop for a single training epoch
@tf.function
def trainInspiredNetwork(inputBatch, outputBatch, generator, discriminator):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        #Generate next simulation iterator using our generator model
        generatedImages = generator(inputBatch, training=True)
        
        #Pass real and fake images to discriminator
        real_output = discriminator((inputBatch, outputBatch), training=True)
        fake_output = discriminator((inputBatch, generatedImages), training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

#Run the training algorithm for multiple epochs
def train(featureData, targetData, epochs, generator, discriminator):
    checkpoint = createTrainingCheckpoint()
    testBatch = next(iter(featureData))
    generate_and_save_images(generator, 0, testBatch)
    for epoch in range(epochs):
        start = time.time()
        for inputBatch, outputBatch in zip(featureData, targetData):
            #print('Blah')
            trainInspiredNetwork(inputBatch, outputBatch, generator, discriminator)

        #Produce images to visualize the training progress
        generate_and_save_images(generator, epoch + 1, testBatch)

        #Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    #Generate after the final epoch
    generate_and_save_images(generator, epochs, testBatch)

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    image = predictions[0].numpy() * 255
    plt.imshow(image.astype(np.uint8))
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    
def createTrainingCheckpoint(checkpoint_dir='./training_checkpoints'):
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(generator_optimizer=generatorOptimizer,
                                     discriminator_optimizer=discriminatorOptimizer,
                                     generator=generator,
                                     discriminator=discriminator)
    return checkpoint

#Adam optimizers for tuning variables in our models during training 
generatorOptimizer = tf.keras.optimizers.Adam(1e-4)
discriminatorOptimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
epochs = 5

#Load data
featureData = getBatchDataset(featuresDir, imageHeight, imageWidth, batch_size)
targetData = getBatchDataset(targetsDir, imageHeight, imageWidth, batch_size)

#Create models
generator = getInspiredGeneratorModel(imageWidth, imageHeight)
discriminator = getInspiredDiscriminatorModel(imageHeight, imageWidth)

print(featureData)
print(featureData.take(10))

#inputBatch = next(iter(featureData))
#generate_and_save_images(generator, 0 + 1, inputBatch)

#Train our models
#train(featureData, targetData, epochs, generator, discriminator)